In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'w'
import random
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
from time import time
def runtime(t0):
    dt = time() - t0
    if dt < 60:
        message = f'{dt:.0f} sec.'
    elif dt < 3600:
        message = f'{dt/60:.0f} min.'
    else:
        h = int(dt//3600)
        m = int((dt%3600) / 60)
        message = f'{h} h. {m} min.'
    print(message)

In [5]:
from matplotlib import animation

In [12]:
"""
Matplotlib Animation Example

author: Jake Vanderplas
email: vanderplas@astro.washington.edu
website: http://jakevdp.github.com
license: BSD
Please feel free to use and modify this, but keep the above information. Thanks!
"""

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-2, 2))
line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 100)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
# anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

# plt.show()

In [52]:

# First set up the figure, the axis, and the plot element we want to animate
xmin, xmax = 0,7
ymax = 2.5
fig = plt.figure(figsize=(15, 5))
ax = plt.axes(xlim=(xmin, xmax), ylim=(-ymax, ymax))
line1, = ax.plot([], [], lw=1, color='r')
line2, = ax.plot([], [], lw=1, color='g')
line3, = ax.plot([], [], lw=2, color='grey')
line4, = ax.plot([], [], lw=2, color='g')

# initialization function: plot the background of each frame
def init():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    return line1, line2, line3, line4,

def wave(x, t, k, w):
    return np.sin(k*x - w*t)

# animation function.  This is called sequentially
def animate(i):
    i /= 10
#     x = np.linspace(xmin, xmax, 200)
    x = np.arange(xmin, xmax, 0.05)
#     y = np.sin(2 * np.pi * (x - 0.01 * i))
    k1, w1 = 5., 5.0/2 #2 * np.pi/.5, 2*np.pi*0.01
    k2, w2 = 3., 4.5/2 #k1*0.95, w1*0.95
    y1 = wave(x, i, k=k1, w=w1)
    y2 = wave(x, i, k=k2, w=w2)
    y3 = 2*wave(x, i, k=(k2-k1)/2, w=(w2-w1)/2)
    
    line1.set_data(x, y1)
    line2.set_data(x, y2)
    line3.set_data(x, y1+y2)
    line4.set_data(x, y3)
    return line1, line2, line3, line4,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=60, blit=True)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()